In [ ]:
import numpy as np
from numpy import linalg as LA
import torch
from torch import linalg as LAT
import scipy
from scipy import linalg as LAS
from scipy.stats import unitary_group
import math
import cv2
import os
from scipy.stats import ortho_group
import scipy.io as sio
import sklearn
from sklearn.model_selection import train_test_split

from tensorly import unfold
rng = np.random.default_rng()

In [ ]:
def import_traffic(numFrames=35,height=50,width=50):
    """
    Collects data from /data/ folder in current working directory, with folder names as labels.
    Inputs:
    -- numFrames=25
    -- height=20,
    -- width=20
    Outputs:
    -- Data as a numpy array (vid number,frame,height,width,color)
    -- Ordered list of video labels
    """
    dataDir = os.getcwd()+"/trafficdb/video/"
    dataNameList = os.listdir(dataDir)
    fileCount = 0
    totalFiles = 0
    ImMs=sio.loadmat(os.getcwd()+'/trafficdb/ImageMaster.mat')        
    class_data = []
    labelList = []
    fname=[]
    totalFiles = np.size(os.listdir(dataDir))
    bigBuffer = np.zeros((totalFiles,numFrames,height,width))
    data_label = []
    #fileCount = 0
    #for filename in os.listdir(dataDir):
    for count in range(254):
        s=ImMs['imagemaster'][0][count][0][0][1][0]
        data_label.append(ImMs['imagemaster'][0][count][0][0][2][0])
        filename=[''.join(s)][0]
        fname.append(filename)
        vid = cv2.VideoCapture(dataDir+"/"+filename+".avi")
        vidFrames = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
        buffer = np.zeros((numFrames,height,width))
        for i in range(numFrames):
            sane, thisFrame = vid.read()                
            if not sane:
                vid.set(cv2.CAP_PROP_POS_FRAMES, 0)
                sane, thisFrame = vid.read()
                if not sane:
                    #print("something broke at",i,filename)
                    break
                
            scale = 1
            height2, width2, channels = thisFrame.shape
            centerX,centerY=int(height2/2),int(width2/2)
            radiusX,radiusY = int(centerX*scale), int(centerY*scale)
            minX,maxX=centerX-radiusX,centerX+radiusX
            minY,maxY=centerY-radiusY,centerY+radiusY
            cropped = thisFrame[minX:maxX, minY:maxY]
                
            buffer[i] = cv2.cvtColor(cv2.resize(cropped,(width,height)),cv2.COLOR_BGR2GRAY)
                
            #label.append(foldername)
        bigBuffer[count]=buffer
        #bigBuffer[fileCount] = buffer
        #fileCount += 1
        vid.release()
    class_data.append(bigBuffer)
        #labelList.append(label)    
    return class_data[0],data_label,fname

trafficdata, traffic_class, filename = import_traffic()

In [ ]:
traffic_class_array = np.empty((254,))
for vid in range(254):
    if traffic_class[vid] == 'heavy':
        traffic_class_array[vid] = 0
    elif traffic_class[vid] == 'medium':
        traffic_class_array[vid] = 1
    elif traffic_class[vid] == 'light':
        traffic_class_array[vid] = 2
videoShuffler = rng.permutation(254)
shuffledClasses = np.take_along_axis(traffic_class_array,videoShuffler,axis=0)
shuffledData = np.take(trafficdata,videoShuffler,axis=0)
heavyTrain,mediumTrain,lightTrain = [],[],[]
testData, testClasses = [],[]
numTestSamples = 76
for vid in shuffledData[-76:,:,:,:]:
    testData.append(vid)
for c in shuffledClasses[-76:]:
    testClasses.append(c)
for vid in range(253-numTestSamples):
    if shuffledClasses[vid] == 0:
        heavyTrain.append(shuffledData[vid,:,:,:])
    elif shuffledClasses[vid] == 1:
        mediumTrain.append(shuffledData[vid,:,:,:])
    elif shuffledClasses[vid] == 2:
        lightTrain.append(shuffledData[vid,:,:,:])

In [ ]:
def my_hosvd(A):
    dim = unfold(A,0)
    dim2 = unfold(A,1)
    dim3 = unfold(A,2)
    
    _,_,vh1 = LAS.svd(dim,full_matrices=False)
    _,_,vh2 = LAS.svd(dim2,full_matrices=False)
    _,_,vh3 = LAS.svd(dim3,full_matrices=False)
    
    return [np.matrix.getH(vh1),np.matrix.getH(vh2),np.matrix.getH(vh3)]

hTX,hTY,hTT = [],[],[]
mTX,mTY,mTT = [],[],[]
lTX,lTY,lTT = [],[],[]
testX,testY,testT = [],[],[]

for vid in heavyTrain:
    fac = my_hosvd(vid)
    hTX.append(fac[0])
    hTY.append(fac[1])
    hTT.append(fac[2])

for vid in mediumTrain:
    fac = my_hosvd(vid)
    mTX.append(fac[0])
    mTY.append(fac[1])
    mTT.append(fac[2])

for vid in lightTrain:
    fac = my_hosvd(vid)
    lTX.append(fac[0])
    lTY.append(fac[1])
    lTT.append(fac[2])

for vid in testData:
    fac = my_hosvd(vid)
    testX.append(fac[0])
    testY.append(fac[1])
    testT.append(fac[2])


In [ ]:
hTrainN = len(heavyTrain)
mTrainN = len(mediumTrain)
lTrainN = len(lightTrain)
testN = len(testData)

hPX,hPY,hPT = np.zeros((hTrainN,hTrainN)),np.zeros((hTrainN,hTrainN)),np.zeros((hTrainN,hTrainN))
mPX,mPY,mPT = np.zeros((mTrainN,mTrainN)),np.zeros((mTrainN,mTrainN)),np.zeros((mTrainN,mTrainN))
lPX,lPY,lPT = np.zeros((lTrainN,lTrainN)),np.zeros((lTrainN,lTrainN)),np.zeros((lTrainN,lTrainN))

for i in range(hTrainN):
    for j in range(hTrainN):
        hPX[i,j] = np.power(np.linalg.norm(hTX[i].T@hTX[j],ord='fro'),2)
        hPY[i,j] = np.power(np.linalg.norm(hTY[i].T@hTY[j],ord='fro'),2)
        hPT[i,j] = np.power(np.linalg.norm(hTT[i].T@hTT[j],ord='fro'),2)
for i in range(mTrainN):
    for j in range(mTrainN):
        mPX[i,j] = np.power(np.linalg.norm(mTX[i].T@mTX[j],ord='fro'),2)
        mPY[i,j] = np.power(np.linalg.norm(mTY[i].T@mTY[j],ord='fro'),2)
        mPT[i,j] = np.power(np.linalg.norm(mTT[i].T@mTT[j],ord='fro'),2)
for i in range(lTrainN):
    for j in range(lTrainN):
        lPX[i,j] = np.power(np.linalg.norm(lTX[i].T@lTX[j],ord='fro'),2)
        lPY[i,j] = np.power(np.linalg.norm(lTY[i].T@lTY[j],ord='fro'),2)
        lPT[i,j] = np.power(np.linalg.norm(lTT[i].T@lTT[j],ord='fro'),2)

In [ ]:
results = []
for testNum in range(testN):
    hVX,hVY,hVT = np.zeros(hTrainN),np.zeros(hTrainN),np.zeros(hTrainN)
    mVX,mVY,mVT = np.zeros(mTrainN),np.zeros(mTrainN),np.zeros(mTrainN)
    lVX,lVY,lVT = np.zeros(lTrainN),np.zeros(lTrainN),np.zeros(lTrainN)

    for i in range(hTrainN):
        hVX[i] = np.power(np.linalg.norm(testX[testNum].T@hTX[i],ord='fro'),2)
        hVY[i] = np.power(np.linalg.norm(testY[testNum].T@hTY[i],ord='fro'),2)
        hVT[i] = np.power(np.linalg.norm(testT[testNum].T@hTT[i],ord='fro'),2)
    for i in range(mTrainN):
        mVX[i] = np.power(np.linalg.norm(testX[testNum].T@mTX[i],ord='fro'),2)
        mVY[i] = np.power(np.linalg.norm(testY[testNum].T@mTY[i],ord='fro'),2)
        mVT[i] = np.power(np.linalg.norm(testT[testNum].T@mTT[i],ord='fro'),2)
    for i in range(lTrainN):
        lVX[i] = np.power(np.linalg.norm(testX[testNum].T@lTX[i],ord='fro'),2)
        lVY[i] = np.power(np.linalg.norm(testY[testNum].T@lTY[i],ord='fro'),2)
        lVT[i] = np.power(np.linalg.norm(testT[testNum].T@lTT[i],ord='fro'),2)

    hYX = 2*np.linalg.inv(hPX + hPX.T)@hVX
    hYY = 2*np.linalg.inv(hPY + hPY.T)@hVY
    hYT = 2*np.linalg.inv(hPT + hPT.T)@hVT
    mYX = 2*np.linalg.inv(mPX + mPX.T)@mVX
    mYY = 2*np.linalg.inv(mPY + mPY.T)@mVY
    mYT = 2*np.linalg.inv(mPT + mPT.T)@mVT
    lYX = 2*np.linalg.inv(lPX + lPX.T)@lVX
    lYY = 2*np.linalg.inv(lPY + lPY.T)@lVY
    lYT = 2*np.linalg.inv(lPT + lPT.T)@lVT

    estXh = testX[testNum]@testX[testNum].T
    estYh = testY[testNum]@testY[testNum].T
    estTh = testT[testNum]@testT[testNum].T
    estXm = testX[testNum]@testX[testNum].T
    estYm = testY[testNum]@testY[testNum].T
    estTm = testT[testNum]@testT[testNum].T
    estXl = testX[testNum]@testX[testNum].T
    estYl = testY[testNum]@testY[testNum].T
    estTl = testT[testNum]@testT[testNum].T

    for i in range(hTrainN):
        estXh -= hTX[i]@hTX[i].T*hYX[i]
        estYh -= hTY[i]@hTY[i].T*hYY[i]
        estTh -= hTT[i]@hTT[i].T*hYT[i]
    estXh = np.linalg.norm(estXh,ord='fro')
    estYh = np.linalg.norm(estYh,ord='fro')
    estTh = np.linalg.norm(estTh,ord='fro')
    
    for i in range(mTrainN):
        estXm -= mTX[i]@mTX[i].T*mYX[i]
        estYm -= mTY[i]@mTY[i].T*mYY[i]
        estTm -= mTT[i]@mTT[i].T*mYT[i]
    estXm = np.linalg.norm(estXm,ord='fro')
    estYm = np.linalg.norm(estYm,ord='fro')
    estTm = np.linalg.norm(estTm,ord='fro')

    for i in range(lTrainN):
        estXl -= lTX[i]@lTX[i].T*lYX[i]
        estYl -= lTY[i]@lTY[i].T*lYY[i]
        estTl -= lTT[i]@lTT[i].T*lYT[i]
    estXl = np.linalg.norm(estXl,ord='fro')
    estYl = np.linalg.norm(estYl,ord='fro')
    estTl = np.linalg.norm(estTl,ord='fro')

    probh = estXh+estYh+estTh
    probm = estXm+estYm+estTm
    probl = estXl+estYl+estTl

    results.append(np.argmin(np.array([probh,probm,probl])))
    print(testNum,results[testNum],testClasses[testNum])

In [ ]:
correct = 0

for i in range(testN):
    if testClasses[i] == results[i]:
        correct += 1
print(correct/76)

In [ ]:
correcth = 0
correctm = 0
correctl = 0
numh = 0
numm = 0
numl = 0
for i in range(testN):
    if testClasses[i] == 0:
      numh += 1
      if results[i] == 0:
          correcth += 1
    if testClasses[i] == 1:
      numm += 1
      if results[i] == 1:
          correctm += 1
    if testClasses[i] == 2:
      numl += 1
      if results[i] == 2:
          correctl += 1
print(correcth/numh)
print(correctm/numm)
print(correctl/numl)

In [ ]:
np.savetxt('traffic_reg_results.csv',results,delimiter=',')
np.savetxt('traffic_reg_classes.csv',testClasses,delimiter=',')